In [86]:
import json
import re
import os
import ast
import csv

In [87]:
def decode_utf8(item):
    if isinstance(item, str):  # If item is a string
        try:
            return item.encode('latin1').decode('utf-8')
        except:
            return item  # Return original if decoding fails
    elif isinstance(item, list):  # If item is a list
        return [decode_utf8(x) for x in item]
    elif isinstance(item, dict):  # If item is a dictionary
        return {key: decode_utf8(value) for key, value in item.items()}
    else:  # If item is neither string, list nor dict (could be int, bool, etc.)
        return item

In [88]:
# read file
# read_file_name = "ffern_123.txt"
# read_file_name = "./txt-accounts-files/pnkkazxz1a.txt"
# read_file_name = "./txt-accounts-files/alocatagram.txt"
read_file_name = "./txt-accounts-files/evgirl.txt"


with open(read_file_name, 'r') as file:
    data = file.read()

data

'b"TikTokApi.video(id=\'7292263544291118342\')"\nb"{\'author\': {\'avatarLarger\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_1080x1080.jpeg?x-expires=1698066000&x-signature=HUx8eY75CgvSqK5ywuoykzHvLIk%3D\', \'avatarMedium\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_720x720.jpeg?x-expires=1698066000&x-signature=e0%2FtmZt7K0%2FDNOQty4gF8YL0thY%3D\', \'avatarThumb\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_100x100.jpeg?x-expires=1698066000&x-signature=JZIQ0r5XtFQ%2FRfp3st9A6ypYZYU%3D\', \'commentSetting\': 0, \'downloadSetting\': 0, \'duetSetting\': 0, \'ftc\': False, \'id\': \'7070728302567490586\', \'isADVirtual\': False, \'isEmbedBanned\': False, \'nickname\': \'EV Girls\', \'openFavorite\': False, \'privateAccount\': False, \'relation\': 0, \'secUid\': \'MS4wLjABAAAABpzU_vDTYU1nfHfKUSRjtRc30yuVXpj_OmhPGRPQIa0yVEaNXQhHNiBRCGFDkMZG\', \

In [89]:
splited_data = data.replace("b'{", 'b"{').split('b"')

arr_data = [spld.replace("\\'", "'") for spld in splited_data]

arr_data

['',
 'TikTokApi.video(id=\'7292263544291118342\')"\n',
 '{\'author\': {\'avatarLarger\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_1080x1080.jpeg?x-expires=1698066000&x-signature=HUx8eY75CgvSqK5ywuoykzHvLIk%3D\', \'avatarMedium\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_720x720.jpeg?x-expires=1698066000&x-signature=e0%2FtmZt7K0%2FDNOQty4gF8YL0thY%3D\', \'avatarThumb\': \'https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_100x100.jpeg?x-expires=1698066000&x-signature=JZIQ0r5XtFQ%2FRfp3st9A6ypYZYU%3D\', \'commentSetting\': 0, \'downloadSetting\': 0, \'duetSetting\': 0, \'ftc\': False, \'id\': \'7070728302567490586\', \'isADVirtual\': False, \'isEmbedBanned\': False, \'nickname\': \'EV Girls\', \'openFavorite\': False, \'privateAccount\': False, \'relation\': 0, \'secUid\': \'MS4wLjABAAAABpzU_vDTYU1nfHfKUSRjtRc30yuVXpj_OmhPGRPQIa0yVEaNXQhHNiBRCGFDkM

In [90]:
def extract_nested_keys(my_dict):

    keys_to_extract = [
        ["author", "uniqueId"],
        ["author", "signature"],
        ["author", "verified"],
        "desc",
        "verified",
        "createTime",
        "digged",
        "duetDisplay",
        "duetEnabled",
        "forFriend",
        "id",
        "itemCommentStatus",
        ["music", "authorName"],
        ["music", "coverLarge"],
        ["music", "duration"],
        ["music", "id"],
        ["music", "original"],
        ["music", "playUrl"],
        ["music", "title"],
    ]

    result = {}
    for key in keys_to_extract:
        if isinstance(key, list):
            nested_result = my_dict
            for nested_key in key:
                if nested_key in nested_result:
                    nested_result = nested_result[nested_key]
                    # print(nested_key)
                else:
                    nested_result = None
                    break
            if nested_result is not None:
                result[str("_".join(key))] = nested_result
        elif key in my_dict:
            result[key] = my_dict[key]
    return result



In [91]:
def extract_hashtags(contents):
    hashtag_arr = []

    for content in contents:
        for key, textExtra_arr in content.items():
            if key == "textExtra":
                for textExtra in textExtra_arr:
                    for key, hashtagName in textExtra.items():
                        if key == "hashtagName":
                            hashtag_arr.append(hashtagName)

        
    # print(hashtag_arr)
    return hashtag_arr

In [92]:
def extract_data(json_data):
    extracted_values = extract_nested_keys(json_data)
    extracted_values["hashtagNames"] = extract_hashtags(json_data["contents"])
    print(str(extracted_values).replace("'",'"').replace("True", "true").replace("False", "false"))
    return extracted_values


In [95]:
# Open the CSV file in append mode
with open("output.csv", "a", newline="", encoding="utf-8") as csvfile:

    for i in range(len(arr_data)):
        if i == 0:
            continue
        elif "TikTokApi.video(" in arr_data[i]:
            continue
        else:
            # try:
                # clean & decode UTF-8
                content = arr_data[i].replace("'", '"')[:-2]
                content = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", content) # reg replace from Don"t to Don't

                # # analyze error
                # pattern1 = r'"title":\s*"(.*?)"'
                # pattern2 = r'"desc":\s*"(.*?)"'
                # updated_string = re.sub(pattern1, '"title": "ignore string error"', content)
                # updated_string = re.sub(pattern2, '"desc": "ignore string error"', updated_string).replace("True", "true").replace("False", "false")

                str_data = ast.literal_eval(content)
                decoded_data = str(decode_utf8(str_data)).replace("'", '"').replace("True", "true").replace("False", "false").replace("\\\\n", " ")
                updated_string = re.sub(r'([A-Za-z])"(?=[A-Za-z])', r"\1'", decoded_data) # reg replace from Don"t to Don't
                # print(updated_string) # print and paste this on the .json file to see if there's an error

                # turn string into object
                json_data = json.loads(updated_string)
                id = json_data["id"]
                # print(id, json_data)

                # # export to check json data
                # file_path = r"./exported-accountpage-json-error-check/" + file_name[:-4] + "-" + id + ".json"
                # with open(file_path, "w", encoding='utf-8') as file:
                #     json.dump(json_data, file, indent=4, ensure_ascii=False)

                extracted_data_json = extract_data(json_data)
                
                # Open the CSV file in write mode
                # file_path = r"./exported-accountpage-json-error-check/" + file_name[:-4] + "-" + id + ".json"
                writer = csv.DictWriter(csvfile, fieldnames=extracted_data_json.keys())
                if i < 3: 
                    writer.writeheader()
                writer.writerow(extracted_data_json)
            # except:
            #     print('error')


{"author_uniqueId": "evgirls.th", "author_signature": "รีวิวรถไฟฟ้า EV ฉบับเข้าใจง่ายๆ", "author_verified": false, "desc": "ซ่อนเนียนมาก😍 #รีวิว #evcar #evgirls #รถยนต์ไฟฟ้า #bmw #i5 #bmwi5 ", "createTime": 1697862417, "digged": false, "duetDisplay": 0, "duetEnabled": true, "forFriend": false, "id": "7292263544291118342", "itemCommentStatus": 0, "music_authorName": "EV Girls", "music_coverLarge": "https://p16-sign-va.tiktokcdn.com/tos-maliva-avt-0068/45df6b464c24e66581a7ef35cb12334d~c5_1080x1080.jpeg?x-expires=1698066000&x-signature=HUx8eY75CgvSqK5ywuoykzHvLIk%3D", "music_duration": 38, "music_id": "7292263577226185478", "music_original": false, "music_playUrl": "https://v16-webapp-prime.tiktok.com/video/tos/useast2a/tos-useast2a-v-27dcd7/ooHZLUNMYzTSFyJuiiGvTMIkwnUvaMlB4AC1Q/?a=1988&ch=0&cr=0&dr=0&er=0&lr=default&cd=0%7C0%7C0%7C0&br=250&bt=125&bti=ODszNWYuMDE6&ft=t2PAl-Inz7T-QapZiyq8Z&mime_type=audio_mpeg&qs=6&rc=PDRpNzxpNmQ2aDg3aGhkNUBpanJ2ZWo5cjpvbjMzNzU8M0A1YzE2Ll8vX2MxYC9jYWFeYSNu